In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay,f1_score, precision_score, recall_score, roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error




In [2]:
# train_data_file_path = r"/content/con_train_data.xlsx"

# Read the Excel file into a DataFrame
con_train_data = pd.read_csv(r"/content/con_train_data.csv")


con_train_data = con_train_data.drop(["Unnamed: 0"], axis=1, errors="ignore")
# train_label_data = train_label_data.drop(["Unnamed: 0"], axis=1, errors="ignore")

# Display the DataFrame
display(con_train_data)
# display(train_label_data)

<ipython-input-2-64b816aed081>:4: DtypeWarning: Columns (7,14) have mixed types. Specify dtype option on import or set low_memory=False.
  con_train_data = pd.read_csv(r"/content/con_train_data.csv")


,residueCount,resolution,structureMolecularWeight,crystallizationTempK,densityMatthews,densityPercentSol,phValue,residueCount.1,resolution.1,structureMolecularWeight.1,...,pdbxDetails_7_ temperature 293K,pdbxDetails_7_ HANGING DROP,pdbxDetails_7_ temperature 298K,pdbxDetails_7_ VAPOR DIFFUSION,pdbxDetails_7_ temperature 277K,pdbxDetails_7_ SITTING DROP,pdbxDetails_7_ temperature 291K,pdbxDetails_7_ temperature 295K,pdbxDetails_7_ temperature 289K,Class label
0,438,1.70,50637.53,298.0,1.95,36.97,6.6,438,1.70,50637.53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TRANSFERASE
1,1324,1.60,148853.81,293.0,2.48,50.38,7.5,1324,1.60,148853.81,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OXIDOREDUCTASE
2,252,1.55,29054.37,NaN,2.03,39.29,6.5,252,1.55,29054.37,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CHAPERONE
3,180,2.30,20674.23,289.0,2.63,52.00,7.5,180,2.30,20674.23,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OXIDOREDUCTASE
4,436,3.50,51456.98,297.0,2.70,54.00,5.6,436,3.50,51456.98,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TRANSFERASE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83570,222,1.70,25022.13,293.0,1.94,36.50,5.6,222,1.70,25022.13,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,LYASE
83571,352,1.85,38873.41,NaN,2.50,50.80,NaN,352,1.85,38873.41,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,COMPLEX (IMMUNOGLOBULIN/HYDROLASE)
83572,0,NaN,13988.69,NaN,1.88,34.72,NaN,0,NaN,13988.69,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,PHOTORECEPTOR
83573,240,2.00,25718.53,NaN,2.80,56.00,6.0,240,2.00,25718.53,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,IMMUNOGLOBULIN


In [3]:
# non_numeric_columns = con_train_data.select_dtypes(exclude=['number']).columns
# print("Columns with non-numeric values:", non_numeric_columns)

con_train_data.drop(39702, inplace=True)


row_39702 = con_train_data.iloc[10067]
print(row_39702)
row_39702_dict = row_39702.to_dict()
print(row_39702_dict)

residueCount                                  2757
resolution                                    4.06
structureMolecularWeight                 307121.34
crystallizationTempK                           NaN
densityMatthews                                NaN
                                        ...       
pdbxDetails_7_ SITTING DROP                    0.0
pdbxDetails_7_ temperature 291K                0.0
pdbxDetails_7_ temperature 295K                0.0
pdbxDetails_7_ temperature 289K                0.0
Class label                        GENE REGULATION
Name: 10067, Length: 114, dtype: object
{'residueCount': 2757, 'resolution': 4.06, 'structureMolecularWeight': 307121.34, 'crystallizationTempK': nan, 'densityMatthews': nan, 'densityPercentSol': nan, 'phValue': nan, 'residueCount.1': 2757, 'resolution.1': 4.06, 'structureMolecularWeight.1': 307121.34, 'crystallizationTempK.1': nan, 'densityMatthews.1': nan, 'densityPercentSol.1': nan, 'phValue.1': nan, 'experimentalTechnique_X-RAY DIF

In [4]:
con_train_data = con_train_data.dropna()

print("Shape of con_train_data : ",con_train_data.shape)

Shape of con_train_data :  (50334, 114)


In [5]:
# Assuming 'Class label' is your target variable
target_variable = 'Class label'

# Separate features and target variable
X = con_train_data.drop(target_variable, axis=1)
y = con_train_data[target_variable]

# reducing the size of data for tesing and memory issues
X =  (X[:20000])
y = list(y[:20000])


# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
imputer = SimpleImputer(missing_values = np.nan,strategy ='median')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train_imputed)
# X_test_scaled = scaler.transform(X_test_imputed)

In [7]:
X_train_imputed_df = pd.DataFrame(X_train_imputed, columns=X_train.columns)
X_test_imputed_df = pd.DataFrame(X_test_imputed, columns=X_test.columns)

# Check for NaN values in the imputed data
print("NaN values in X_train_imputed:")
display(X_train_imputed_df.isnull().sum())

print("\nNaN values in X_test_imputed:")
display(X_test_imputed_df.isnull().sum())

NaN values in X_train_imputed:


residueCount                       0
resolution                         0
structureMolecularWeight           0
crystallizationTempK               0
densityMatthews                    0
                                  ..
pdbxDetails_7_ temperature 277K    0
pdbxDetails_7_ SITTING DROP        0
pdbxDetails_7_ temperature 291K    0
pdbxDetails_7_ temperature 295K    0
pdbxDetails_7_ temperature 289K    0
Length: 113, dtype: int64


NaN values in X_test_imputed:


residueCount                       0
resolution                         0
structureMolecularWeight           0
crystallizationTempK               0
densityMatthews                    0
                                  ..
pdbxDetails_7_ temperature 277K    0
pdbxDetails_7_ SITTING DROP        0
pdbxDetails_7_ temperature 291K    0
pdbxDetails_7_ temperature 295K    0
pdbxDetails_7_ temperature 289K    0
Length: 113, dtype: int64

In [8]:
print("Any NaN values in X_train_imputed:", np.isnan(X_train_imputed).any().any())
print("Any NaN values in X_test_imputed:", np.isnan(X_test_imputed).any())



Any NaN values in X_train_imputed: False
Any NaN values in X_test_imputed: False


In [ ]:
# rfc = RandomForestClassifier(n_estimators=50, random_state=42,max_features = 0.6,max_samples=0.5,max_depth=2)  # You can adjust the number of estimators

# # Fit the model on the imputed training data
# rfc.fit(X_train_imputed, y_train)

# # Predict on the imputed test data
# y_pred_imputed = rfc.predict(X_train_imputed)

# # Calculate accuracy and other metrics for imputed data
# accuracy_imputed = accuracy_score(y_test, y_pred_imputed)
# classification_rep_imputed = classification_report(y_test, y_pred_imputed, zero_division=1)

# # Print the results for imputed data
# print(f"Accuracy (imputed): {accuracy_imputed:.4f}")
# print("Classification Report (imputed):\n", classification_rep_imputed)

## After Hypertuning

In [9]:
rfc_model = RandomForestClassifier(n_estimators=120, min_samples_split=5, min_samples_leaf=1, max_samples =0.75, max_features=0.6,
max_depth=None, bootstrap =True)  # You can adjust the number of estimators

# Fit the model on the imputed training data
rfc_model.fit(X_train_imputed, y_train)

# Predict on the imputed test data
y_pred_imputed = rfc_model.predict(X_test_imputed)

# Calculate accuracy and other metrics for imputed data
accuracy_imputed = accuracy_score(y_test, y_pred_imputed)
classification_rep_imputed = classification_report(y_test, y_pred_imputed, zero_division=1)

# Print the results for imputed data
print(f"Accuracy (imputed): {accuracy_imputed:.4f}")
print("Classification Report (imputed):\n", classification_rep_imputed)

Accuracy (imputed): 0.3458
Classification Report (imputed):
                                          precision    recall  f1-score   support

          ACETYLCHOLINE-BINDING PROTEIN       1.00      0.00      0.00         1
                               ALLERGEN       0.00      0.00      0.00         4
                   ANTIBIOTIC INHIBITOR       1.00      0.00      0.00         1
                     ANTIFREEZE PROTEIN       1.00      0.00      0.00         1
                     ANTIFUNGAL PROTEIN       0.00      1.00      0.00         0
                  ANTIMICROBIAL PROTEIN       0.00      0.00      0.00         8
                      ANTITUMOR PROTEIN       1.00      0.00      0.00         1
                      ANTIVIRAL PROTEIN       1.00      0.00      0.00         4
                              APOPTOSIS       0.75      0.23      0.35        13
                    APOPTOSIS INHIBITOR       1.00      0.00      0.00         2
                APOPTOSIS/IMMUNE SYSTEM       1

In [10]:
print(f"Accuracy (imputed): {accuracy_imputed:.4f}")

Accuracy (imputed): 0.3458


## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate the confusion matrix
conf_matrix_imputed = confusion_matrix(y_test, y_pred_imputed)

# # Display the confusion matrix using Seaborn
# plt.figure(figsize=(8, 6))
# sns.heatmap(conf_matrix_imputed, annot=True, fmt="d", cmap="Blues", linewidths=.5, cbar=False)
# plt.title("Confusion Matrix (Imputed)")
# plt.xlabel("Predicted Labels")
# plt.ylabel("True Labels")
# plt.show()
cm_display = ConfusionMatrixDisplay(confusion_matrix = conf_matrix_imputed)
cm_display.plot()

## Cross Validation Running

In [15]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
import numpy as np

# Define the cross-validation strategy (StratifiedKFold is commonly used for classification)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Create a new RandomForestClassifier instance with the same parameters
rfc_model_cv = RandomForestClassifier(n_estimators=120, min_samples_split=5, min_samples_leaf=1,
                                      max_samples=0.75, max_features=0.6, max_depth=None, bootstrap=True)

# Perform cross-validation and get accuracy scores
cv_scores = cross_val_score(rfc_model_cv, X_train_imputed, y_train, cv=cv, scoring='accuracy')

# Print cross-validation results
print("Cross-Validation Scores:")
print(cv_scores)
print(f"Mean Accuracy: {np.mean(cv_scores):.4f}")
print(f"Standard Deviation: {np.std(cv_scores):.4f}")


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross-Validation Scores:
[0.3275    0.31375   0.33125   0.3203125 0.3171875]
Mean Accuracy: 0.3220
Standard Deviation: 0.0065


# Random Search Cv for Random Forest Classifer
(Just for finding Hyperparameters)

In [77]:
# Number of trees in random forest
n_estimators = [20,60,100,120]

# Number of features to consider at every split
max_features = [0.2,0.6]

# Maximum number of levels in tree
max_depth = [2,8,None]

# # Number of samples
# max_samples = [0.5,0.75,1.0]

# Bootstrap samples
bootstrap = [True,False]

# Minimum number of samples required to split a node
min_samples_split = [2, 5]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]

In [78]:
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,

              'bootstrap':bootstrap,
              'min_samples_split':min_samples_split,
              'min_samples_leaf':min_samples_leaf
             }
print(param_grid)

              # 'max_samples':max_samples,

{'n_estimators': [20, 60, 100, 120], 'max_features': [0.2, 0.6], 'max_depth': [2, 8, None], 'bootstrap': [True, False], 'min_samples_split': [2, 5], 'min_samples_leaf': [1, 2]}


In [69]:
from sklearn.model_selection import RandomizedSearchCV

rf = RandomForestClassifier()
rf_grid = RandomizedSearchCV(estimator = rf,
                       param_distributions = param_grid,
                       cv = 5,
                       verbose=2,
                       n_jobs = -1)

In [70]:
rf_grid.fit(X_train_imputed, y_train)


Fitting 5 folds for each of 10 candidates, totalling 50 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:378: FitFailedWarning: 
25 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
25 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py", line 397, in fit
    raise ValueError(
ValueErr

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [2, 8, None],
                                        'max_features': [0.2, 0.6],
                                        'max_samples': [0.5, 0.75, 1.0],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': [20, 60, 100, 120]},
                   verbose=2)

In [71]:
rf_grid.best_params_

{'n_estimators': 120,
 'min_samples_split': 5,
 'min_samples_leaf': 1,
 'max_samples': 0.75,
 'max_features': 0.6,
 'max_depth': None,
 'bootstrap': True}

In [74]:
rf_grid.best_score_


0.24975

In [ ]:
# {'n_estimators': 120,
#  'min_samples_split': 5,
#  'min_samples_leaf': 1,
#  'max_samples': 0.75,
#  'max_features': 0.6,
#  'max_depth': None,
#  'bootstrap': True}

# GridSearchCV

In [ ]:
from sklearn.svm import SVC

# Define the parameter grid for SVM
param_grid_svm = {
    'C': [0.1, 1, 10, 100],              # Regularization parameter
    'kernel': ['linear', 'rbf', 'poly'], # Kernel type
    'gamma': ['scale', 'auto', 0.1, 1],  # Kernel coefficient for 'rbf' and 'poly'
    'degree': [2, 3, 4],                  # Degree of the polynomial kernel
    'coef0': [0, 1],                      # Independent term in the kernel function ('poly' and 'sigmoid')
    'shrinking': [True, False],           # Whether to use the shrinking heuristic
    'class_weight': [None, 'balanced'],   # Weights associated with classes
}

# Create an SVM model
svm_model = SVC()

# Create a GridSearchCV object for SVM
grid_svm = GridSearchCV(estimator=svm_model, param_grid=param_grid_svm, cv=5, verbose=2, n_jobs=-1)

# Fit the model on your data
# Assuming X_train and y_train are your training data
grid_svm.fit(X_train, y_train)

# Print the best parameters and corresponding accuracy
print("Best Parameters for SVM:", grid_svm.best_params_)
print("Best Accuracy for SVM:", grid_svm.best_score_)

In [80]:


from sklearn.model_selection import GridSearchCV

rf_grid = GridSearchCV(estimator = rf,
                       param_grid = param_grid,
                       cv = 5,
                       verbose=2,
                       n_jobs = -1)



In [81]:
rf_grid.fit(X_train_imputed, y_train)


Fitting 5 folds for each of 192 candidates, totalling 960 fits


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


KeyboardInterrupt: ignored